# Fireup graphlab

In [1]:
import graphlab

In [2]:
graphlab.set_runtime_config('GRAPHLAB_DEFAULT_NUM_GRAPH_LAMBDA_WORKERS',4)

This non-commercial license of GraphLab Create for academic use is assigned to akash.kashyp@gmail.com and will expire on July 28, 2017.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1472358930.log


## Load the data

In [3]:
image_train = graphlab.SFrame('../Datasets/DeepLearning/image_train_data/')
image_test = graphlab.SFrame('../Datasets/DeepLearning/image_test_data/')

In [4]:
graphlab.canvas.set_target('ipynb')

In [5]:
image_train.head()

Columns:
	id	int
	image	Image
	label	str
	deep_features	array
	image_array	array

Rows: 10

Data:
+-----+----------------------+------------+-------------------------------+
|  id |        image         |   label    |         deep_features         |
+-----+----------------------+------------+-------------------------------+
|  24 | Height: 32 Width: 32 |    bird    | [0.242871761322, 1.0954537... |
|  33 | Height: 32 Width: 32 |    cat     | [0.525087952614, 0.0, 0.0,... |
|  36 | Height: 32 Width: 32 |    cat     | [0.566015958786, 0.0, 0.0,... |
|  70 | Height: 32 Width: 32 |    dog     | [1.12979578972, 0.0, 0.0, ... |
|  90 | Height: 32 Width: 32 |    bird    | [1.71786928177, 0.0, 0.0, ... |
|  97 | Height: 32 Width: 32 | automobile | [1.57818555832, 0.0, 0.0, ... |
| 107 | Height: 32 Width: 32 |    dog     | [0.0, 0.0, 0.220677852631,... |
| 121 | Height: 32 Width: 32 |    bird    | [0.0, 0.23753464222, 0.0, ... |
| 136 | Height: 32 Width: 32 | automobile | [0.0, 0.0, 0.0, 0.0, 0.0, ... |
| 138 | Height: 32 Width: 32 |    bird    | [0.658935725689, 0.0, 0.0,... |
+-----+----------------------+------------+-------------------------------+
+-------------------------------+
|          image_array          |
+-------------------------------+
| [73.0, 77.0, 58.0, 71.0, 6... |
| [7.0, 5.0, 8.0, 7.0, 5.0, ... |
| [169.0, 122.0, 65.0, 131.0... |
| [154.0, 179.0, 152.0, 159.... |
| [216.0, 195.0, 180.0, 201.... |
| [33.0, 44.0, 27.0, 29.0, 4... |
| [97.0, 51.0, 31.0, 104.0, ... |
| [93.0, 96.0, 88.0, 102.0, ... |
| [35.0, 59.0, 53.0, 36.0, 5... |
| [205.0, 193.0, 195.0, 200.... |
+-------------------------------+
[10 rows x 5 columns]

# Computing the least common category

In [6]:
image_train['label'].sketch_summary()


+------------------+-------+----------+
|       item       | value | is exact |
+------------------+-------+----------+
|      Length      |  2005 |   Yes    |
| # Missing Values |   0   |   Yes    |
| # unique values  |   4   |    No    |
+------------------+-------+----------+

Most frequent items:
+-------+------------+-----+-----+------+
| value | automobile | cat | dog | bird |
+-------+------------+-----+-----+------+
| count |    509     | 509 | 509 | 478  |
+-------+------------+-----+-----+------+


In [7]:
dog_category = image_train[image_train['label']=='dog']
bird_category = image_train[image_train['label']=='bird']
cat_category = image_train[image_train['label']=='cat']
automobile_category = image_train[image_train['label']=='automobile']

## creating nearest neighbour model for each category

In [8]:
dog_model = graphlab.nearest_neighbors.create(dog_category,features=['deep_features'],label='id')
cat_model = graphlab.nearest_neighbors.create(cat_category,features=['deep_features'],label='id')
automobile_model = graphlab.nearest_neighbors.create(automobile_category,features=['deep_features'],label='id')
bird_model = graphlab.nearest_neighbors.create(bird_category,features=['deep_features'],label='id')

Starting brute force nearest neighbors model training.

Starting brute force nearest neighbors model training.

Starting brute force nearest neighbors model training.

Starting brute force nearest neighbors model training.

In [9]:
image_test[0:1]

Columns:
	id	int
	image	Image
	label	str
	deep_features	array
	image_array	array

Rows: 1

Data:
+----+----------------------+-------+-------------------------------+
| id |        image         | label |         deep_features         |
+----+----------------------+-------+-------------------------------+
| 0  | Height: 32 Width: 32 |  cat  | [1.13469004631, 0.0, 0.0, ... |
+----+----------------------+-------+-------------------------------+
+-------------------------------+
|          image_array          |
+-------------------------------+
| [158.0, 112.0, 49.0, 159.0... |
+-------------------------------+
[1 rows x 5 columns]

In [12]:
knn_cats = cat_model.query(image_test[0:1])

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 27.282ms     |

| Done         |         | 100         | 105.234ms    |

+--------------+---------+-------------+--------------+

In [13]:
knn_dogs = dog_model.query(image_test[0:1])

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 27.797ms     |

| Done         |         | 100         | 103.371ms    |

+--------------+---------+-------------+--------------+

In [15]:
knn_cats

Columns:
	query_label	int
	reference_label	int
	distance	float
	rank	int

Rows: 5

Data:
+-------------+-----------------+---------------+------+
| query_label | reference_label |    distance   | rank |
+-------------+-----------------+---------------+------+
|      0      |      16289      |  34.623719208 |  1   |
|      0      |      45646      | 36.0068799284 |  2   |
|      0      |      32139      | 36.5200813436 |  3   |
|      0      |      25713      | 36.7548502521 |  4   |
|      0      |       331       | 36.8731228168 |  5   |
+-------------+-----------------+---------------+------+
[5 rows x 4 columns]

In [16]:
knn_dogs

Columns:
	query_label	int
	reference_label	int
	distance	float
	rank	int

Rows: 5

Data:
+-------------+-----------------+---------------+------+
| query_label | reference_label |    distance   | rank |
+-------------+-----------------+---------------+------+
|      0      |      16976      | 37.4642628784 |  1   |
|      0      |      13387      | 37.5666832169 |  2   |
|      0      |      35867      | 37.6047267079 |  3   |
|      0      |      44603      | 37.7065585153 |  4   |
|      0      |       6094      | 38.5113254907 |  5   |
+-------------+-----------------+---------------+------+
[5 rows x 4 columns]

In [20]:
image_train[image_train['id']==16289]['image'].show()

In [21]:
image_train[image_train['id']==16976]['image'].show()

In [22]:
cat_image_test = image_test[image_test['label'] == 'cat']
dog_image_test = image_test[image_test['label'] == 'dog']
bird_image_test = image_test[image_test['label'] == 'bird']
car_image_test = image_test[image_test['label'] == 'automobile']

In [24]:

dog_cat_neighbors = cat_model.query(dog_image_test, k=1)
dog_bird_neighbors = bird_model.query(dog_image_test, k=1)
dog_car_neighbors = automobile_model.query(dog_image_test, k=1)
dog_dog_neighbors = dog_model.query(dog_image_test, k=1)

Starting blockwise querying.

max rows per data block: 4348

number of reference data blocks: 4

number of query data blocks: 1

+--------------+---------+-------------+--------------+

...

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 1000         | 128000  | 25.1473     | 435.038ms    |

| Done         | 509000  | 100         | 511.601ms    |

+--------------+---------+-------------+--------------+

In [25]:
dog_car_neighbors.head()

Columns:
	query_label	int
	reference_label	int
	distance	float
	rank	int

Rows: 10

Data:
+-------------+-----------------+---------------+------+
| query_label | reference_label |    distance   | rank |
+-------------+-----------------+---------------+------+
|      0      |      33859      | 41.9579761457 |  1   |
|      1      |       2046      | 46.0021331807 |  1   |
|      2      |      19594      | 42.9462290692 |  1   |
|      3      |      11000      | 41.6866060048 |  1   |
|      4      |      19594      | 39.2269664935 |  1   |
|      5      |      49314      | 40.5845117698 |  1   |
|      6      |      40822      | 45.1067352961 |  1   |
|      7      |      44997      | 41.3221140974 |  1   |
|      8      |      33859      | 41.8244654995 |  1   |
|      9      |      33859      | 45.4976929401 |  1   |
+-------------+-----------------+---------------+------+
[10 rows x 4 columns]

In [26]:

dog_distances = graphlab.SFrame({'dog-car': dog_car_neighbors['distance'], 'dog-bird': dog_bird_neighbors['distance'], 'dog-cat': dog_car_neighbors['distance'], 'dog-dog': dog_dog_neighbors['distance']})

In [27]:

dog_distances.head()

Columns:
	dog-bird	float
	dog-car	float
	dog-cat	float
	dog-dog	float

Rows: 10

Data:
+---------------+---------------+---------------+---------------+
|    dog-bird   |    dog-car    |    dog-cat    |    dog-dog    |
+---------------+---------------+---------------+---------------+
| 41.7538647304 | 41.9579761457 | 41.9579761457 | 33.4773590373 |
| 41.3382958925 | 46.0021331807 | 46.0021331807 | 32.8458495684 |
| 38.6157590853 | 42.9462290692 | 42.9462290692 | 35.0397073189 |
| 37.0892269954 | 41.6866060048 | 41.6866060048 | 33.9010327697 |
|  38.272288694 | 39.2269664935 | 39.2269664935 | 37.4849250909 |
| 39.1462089236 | 40.5845117698 | 40.5845117698 |  34.945165344 |
|  40.523040106 | 45.1067352961 | 45.1067352961 | 39.0957278345 |
| 38.1947918393 | 41.3221140974 | 41.3221140974 | 37.7696131032 |
| 40.1567131661 | 41.8244654995 | 41.8244654995 | 35.1089144603 |
| 45.5597962603 | 45.4976929401 | 45.4976929401 | 43.2422832585 |
+---------------+---------------+---------------+---------------+
[10 rows x 4 columns]

In [28]:
def is_dog_correct(row):
    return row['dog-dog'] < row['dog-cat'] and row['dog-dog'] < row['dog-car'] and row['dog-dog'] < row['dog-bird'] 
dog_distances['predict'] = dog_distances.apply(is_dog_correct)

In [29]:
dog_distances['predict'].sketch_summary()


+--------------------+----------------+----------+
|        item        |     value      | is exact |
+--------------------+----------------+----------+
|       Length       |      1000      |   Yes    |
|        Min         |      0.0       |   Yes    |
|        Max         |      1.0       |   Yes    |
|        Mean        |     0.879      |   Yes    |
|        Sum         |     879.0      |   Yes    |
|      Variance      |    0.106359    |   Yes    |
| Standard Deviation | 0.326127275768 |   Yes    |
|  # Missing Values  |       0        |   Yes    |
|  # unique values   |       2        |    No    |
+--------------------+----------------+----------+

Most frequent items:
+-------+-----+-----+
| value |  1  |  0  |
+-------+-----+-----+
| count | 879 | 121 |
+-------+-----+-----+

Quantiles: 
+-----+-----+-----+-----+-----+-----+-----+-----+------+
|  0% |  1% |  5% | 25% | 50% | 75% | 95% | 99% | 100% |
+-----+-----+-----+-----+-----+-----+-----+-----+------+
| 0.0 | 0.0 | 0.0 | 

In [30]:
dog_distances['predict'].sum()
dog_distances.show()

In [31]:
cat_cat_neighbors = cat_model.query(cat_image_test, k=1)
cat_bird_neighbors = bird_model.query(cat_image_test, k=1)
cat_car_neighbors = automobile_model.query(cat_image_test, k=1)
cat_dog_neighbors = dog_model.query(cat_image_test, k=1)

Starting blockwise querying.

max rows per data block: 4348

number of reference data blocks: 4

number of query data blocks: 1

+--------------+---------+-------------+--------------+

...

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 1000         | 127000  | 24.9509     | 389.02ms     |

| Done         | 509000  | 100         | 449.103ms    |

+--------------+---------+-------------+--------------+

In [32]:
cat_distances = graphlab.SFrame({'cat-car': cat_car_neighbors['distance'], 'cat-bird': cat_bird_neighbors['distance'], 'cat-cat': cat_cat_neighbors['distance'], 'cat-dog': cat_dog_neighbors['distance']})

In [33]:
cat_distances.head()

Columns:
	cat-bird	float
	cat-car	float
	cat-cat	float
	cat-dog	float

Rows: 10

Data:
+---------------+---------------+---------------+---------------+
|    cat-bird   |    cat-car    |    cat-cat    |    cat-dog    |
+---------------+---------------+---------------+---------------+
|  38.074265869 | 39.6710582792 |  34.623719208 | 37.4642628784 |
| 36.3674024138 | 43.0089056688 | 33.8680579302 | 29.3472319585 |
| 35.3039394947 | 38.6010006604 | 32.4615168902 | 32.2599640475 |
| 38.8944029601 | 39.3566307091 | 35.7708210254 | 35.3852085188 |
| 34.2820409875 | 38.3572372618 | 31.1577686417 | 30.0442985088 |
| 44.5352170178 | 42.0904793181 | 41.3986035847 | 35.4741000424 |
| 34.0290595084 | 39.0520251253 | 30.9894594959 | 32.5845275226 |
| 39.0236924983 | 39.3058645069 | 37.0814607387 | 37.6502852614 |
| 40.8334054297 | 43.0248129799 | 39.9883863688 | 36.9801353512 |
| 40.1258835601 | 45.6749176426 | 39.7076633097 | 41.1259410707 |
+---------------+---------------+---------------+---------------+
[10 rows x 4 columns]

In [34]:
def is_cat_correct(row):
    return row['cat-cat'] < row['cat-dog'] and row['cat-cat'] < row['cat-car'] and row['cat-cat'] < row['cat-bird'] 
cat_distances['predict'] = cat_distances.apply(is_cat_correct)

In [35]:
cat_distances['predict'].sum()

548

# retrieving images using deep features

In [36]:
knn_model = graphlab.nearest_neighbors.create(image_train,features=['deep_features'],
                                             label='id')

Starting brute force nearest neighbors model training.

In [37]:
graphlab.canvas.set_target('ipynb')
cat = image_train[18:19]
cat['image'].show()

In [38]:
knn_model.query(cat)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 16.809ms     |

| Done         |         | 100         | 233.073ms    |

+--------------+---------+-------------+--------------+

Columns:
	query_label	int
	reference_label	int
	distance	float
	rank	int

Rows: 5

Data:
+-------------+-----------------+---------------+------+
| query_label | reference_label |    distance   | rank |
+-------------+-----------------+---------------+------+
|      0      |       384       |      0.0      |  1   |
|      0      |       6910      | 36.9403137951 |  2   |
|      0      |      39777      | 38.4634888975 |  3   |
|      0      |      36870      | 39.7559623119 |  4   |
|      0      |      41734      | 39.7866014148 |  5   |
+-------------+-----------------+---------------+------+
[5 rows x 4 columns]

In [39]:
def get_images_from_ids(query_result):
    return image_train.filter_by(query_result['reference_label'],'id')

In [40]:
cat_neighbors = get_images_from_ids(knn_model.query(cat))

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 16.047ms     |

| Done         |         | 100         | 222.163ms    |

+--------------+---------+-------------+--------------+

In [41]:
cat_neighbors['image'].show()

In [43]:
car = image_train[8:9]
car['image'].show()

In [44]:
get_images_from_ids(knn_model.query(car))['image'].show()

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 12.539ms     |

| Done         |         | 100         | 232.711ms    |

+--------------+---------+-------------+--------------+